In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
np.set_printoptions(legacy = '1.25')

import diempy as diem
import time

import sys
import os

%matplotlib widget

# Prepare Data for `diem` analysis

## Convert vcf file to diem input files

In [ ]:
vcfFile = './data/iphiclides_example.vcf.gz'

In [ ]:
os.system('vcf2diem '+vcfFile)

## Update Ploidy information & Recombination rates

vcf2diem generates a meta data file that provides information on chromosome lengths, their relative recombination rates, and the ploidy of each chromosome for each individual.  The ploidy information is used during the polarisation step, and the information about relative recombination rates is used when smoothing the data post-polarisation.  

vcf2diem outputs a default ploidy of 2 and a relative recombination rate of 1.0. This must be updated by the user according to their own dataset. 

**Ploidy**

By default, vcf2diem assigns a ploidy of 2 across all individuals and chromosomes.  However, for some chromosomes, this will not be the case.  For example, in an XY system, the male is haploid for the X chromosome. In a ZW system like that of this iphiclides dataset, females are haploid for the Z chromosome.  In our example, we provide a file to update the individual ploidies for this specific chromosome.


**Relative Recombination Rates**

vcf2diem outputs a default *relative* recombination rate of 1.0. This essentially assumes that every chromosome experiences the same amount of recombination, regardless of the length and regardless of whether it is a sex chromosome.

This is approximately true for Lepidopterans, like iphiclides, where each chromosome undergoes one recombination event during meiosis.  However, even in this case, we must account for some nuance. 

In Lepidoptera, recombination occurs normally in males, the homogametic sex (ZZ), but is completely absent in females, the heterogametic sex (ZW).

An autosome spends 1/2 of its time in males, so undergoes recombination in 1/2 of its meiotic events. In contrast, the Z chromsome spends 2/3 of it's time in males, so experiences recombination in 2/3 of its meiotic events. Therefore, the relative rate of recombination is higher on the Z chromosome compared to an autosome: (2/3) / (1/2) = 4/3.

We can update this information in the meta data file to account for this difference when smoothing. The file 'relative_rec_rates.txt' shows an example of how this is done. Note that only chromosomes with a rate that is not 1 need to be specified.


In [ ]:
metaRawFile = './data/iphiclides_example.vcf.gz.diem_meta.bed'

ploidyFile = './user_files/iphiclides_ploidy_update.txt'
recFile = './user_files/relative_rec_rates.txt'

metaCorrected = './data/iphiclides_example.vcf.gz.diem_meta_corrected.bed'

In [ ]:
diem.update_meta(metaRawFile,metaCorrected,ploidyFilePath=ploidyFile,recFilePath=recFile)

**other ways relative recombination rates may differ among chromosomes**

The relative rate of recombination across chromosomes will vary substantially among organisms. For example, in some organisms, recombination occurs at a per-base-pair rate. In that case, longer chromosomes will have a relatively higher total rate of recombination.

To account for this, the user could specify a recombination rate that scales with chromosome size.  For example, if chromosome i has length Li:

chr 1 --> L1 --> L1/L1 = 1.0

chr 2 --> L2 --> L2/L1

chr 3 --> L3 --> L3/L1

# Reading data into diemPy

We now have our inbut bed file and our *corrected* meta data file.  

We can read that in as a 'raw' or 'unpolarized' DiemType object.

The DiemType is a class which holds the data and provides the core functionality of diem. Refer to documentation for full details.

Here, we will simply show how it is used in a standard workflow. 

In [ ]:
bedInFile = './data/iphiclides_example.vcf.gz.diem_input.bed'
metaCorrected = './data/iphiclides_example.vcf.gz.diem_meta_corrected.bed'


In [ ]:
dRaw = diem.read_diem_bed(bedInFile,metaCorrected)


# Polarization

To polarize the data, we simply call the polarize() function of the DiemType object we created above.

Note:
- that it defaults to a parallel evaluation using the full number of available cores. We can change the number of cores manually using the 'ncores=' optional argument.  If n=1, it performs a serial polarization of the data.
- it returns a separate *copy* of the DiemType object that is now polarized. That is, the raw object is not changed.

In [ ]:
dPol = dRaw.polarize(ncores=4)

We can now save this to a bed file which has the same format as the input file but added information from polarization

In [ ]:
bedOutFile = './output/diem_output.bed'
diem.write_polarized_bed(bedInFile,bedOutFile,dPol)


We can also save this directly as a diemtype object. 

It uses less memory, and it is how we will work with the data moving forward.

However, it is only useable within the diem environment. 

In [ ]:
polarizedDiemTypeOutFile = './output/example_polarized.diemtype'

diem.save_DiemType(dPol,polarizedDiemTypeOutFile)


If we have already polarized the data, we can load the data back in either way:

In [ ]:
dPol = diem.load_DiemType(polarizedDiemTypeOutFile) 
#or
dPol = diem.read_diem_bed(bedOutFile,metaCorrected)

# Hybrid Indices and Sorting 

the `diem` algorithm sorts individuals by hybrid index and seeks to maximize the differences between two sets of individuals on either side of a barrier to gene flow.

although the hybrid indices are calculated on polarization, the individuals are not sorted by hybrid index as we can see here:

In [ ]:
fig,ax = plt.subplots(figsize = (6,4))
ax.plot(dPol.HIs,marker='.',)
plt.ylim(0,1)
ax.set_ylabel('HI')
ax.set_xlabel('individual idx')
plt.show()

We can use the sort() function to sort the individuals. 

In [ ]:
dPol.sort()

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
ax.plot(dPol.HIs,marker='.',)
plt.ylim(0,1)
ax.set_ylabel('HI')
ax.set_xlabel('individual idx')
plt.show()

# Plotting a Painting

Now that we have polarized and sorted the data, let's have a look at it using the plot_painting() function:

In [ ]:
diem.plot_painting?

In [ ]:
#here we are plotting chromosome 9
diem.plot_painting(dPol.DMBC[9],names=dPol.indNames)


Simply for illustration, let's compare this to the null polarization that diem started with:

In [ ]:
diem.plot_painting(diem.diemtype.flip_polarity(dPol.DMBC[9],dPol.initialPolByChr[9]),names=dPol.indNames)

# The Diagnostic Index (DI) and Thresholding

## The DI distribution 

Here we plot the distribution of diagnostic indices across the data.  For each of the final marker configurations, the diagnostic index tells use how well the marker matches does at explaining our data.  The least-negative value is the best.

In teal, we plot a histogram of the DI values for each unique marker.

In magenta, the counts of each marker are taken into account.

In [ ]:
DIValsWithRepeats = np.hstack(dPol.DIByChr)
DIValsUnique = np.unique(DIValsWithRepeats)

fig,ax = plt.subplots(figsize=(5,4))
(counts,bins,patches) = ax.hist(DIValsWithRepeats,bins=500,color='darkmagenta')
ax.hist(DIValsUnique,bins=bins,color='lightseagreen')
plt.yscale('log')
plt.show()

We provide a function that will characterize the marker configurations, and provide information about each:  the count, the DI, the number of differences from the 'ideal marker' (the one with the highest DI) and how  many of the individuals have missing data for that marker. It returns a pandas dataframe.

This could be useful for exploring the data in greater detail when deciding on a threshold value in later steps.

**note** that this could be memory and time intensive to generate if there are many marker types.  We recommend characterizing only a subset of the most-informative markers to be on the safe side.  It is those markers that we will retain through thresholding later.



In [ ]:
dfMarkers = diem.characterize_markers(dPol.apply_threshold(-50))
dfMarkers

## applying the final threshold chosen

The DI distribution for every dataset will be unique.  In the **Visualization** section of the tutorial, we provide further tools and guidance on choosing a threshold. 

For illustration here, we pick a threshold of -30.  

Thresholding returns a copy of the DiemType object, and it prints the proportion of markers retained for each chromsoome.

In [ ]:
dThresh = dPol.apply_threshold(-30)

After thresholding, there will be fewer markers present, and those that remain have configurations which better support the genetic barrier. In effect, the data looks cleaner. We can see this by plotting chromosome paintings with markers in their physical positions, below, both before and after thresholding. 

In [ ]:
diem.plot_painting_with_positions(dPol.DMBC[9],dPol.posByChr[9],names=dPol.indNames)
diem.plot_painting_with_positions(dThresh.DMBC[9],dThresh.posByChr[9],names=dPol.indNames)

Below, we compare the hybrid indices before (blue) and after (red) thresholding: 

We can see that the separation of individuals is clearer now, and their hybrid indices have also changed.

We will not do this for now, but we could resort the individuals by HI by calling *dThresh.sort()*

In [ ]:
fig,ax = plt.subplots(figsize = (6,4))
ax.plot(dPol.HIs,marker='.',color='b')
ax.plot(dThresh.HIs,marker='.',color='r')
ax.set_ylim(0,1)
ax.set_ylabel('HI')
ax.set_xlabel('individual idx')
plt.show()

# Smoothing the Data

After thresholding, we will see much clearer tracts representing shared ancestry with respect to the barrier.  However, there will be some bits of missing data, mis-called sites, etc. that will break up the tracts. We want to smooth this over for later analyses. We also care about tracts generated by cross-over recombination, but gene conversion events could also cause short changes in state and break up our tracts.  Therefore, we want to smooth over this noise and this signal in the data.

In the manuscript **(insert link when available)**, we explain the Laplacian smoothing kernel and how this smoothing is done on a recombination-based (map) distance rather than physical base pair position.

The default behaviour of DiemPy is to normalize the physical distances by chromosome length and use a global smoothing scale parameter across all chromosomes. This assumption can be relaxed, and we discuss this later in the **Advanced Smoothing** section.


Here, we use the default global scale, and to pick a smoothing scale, we examine how the data changes for different values.  Below, we calculate the number of sites that change across the whole dataset using the laplacian smoothign kernel:

In [ ]:
scalesToTry = [1e-8,5e-8,1e-7,5e-7,1e-6,5e-6,1e-5,5e-5,1e-4,5e-4]
sitesDiffByScale = []
for scale in scalesToTry:
    dSmoothedTest = dThresh.smooth(scale)
    kdiffs = diem.count_site_differences(dSmoothedTest.DMBC,dThresh.DMBC)
    sitesDiffByScale.append(kdiffs)

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
ax.plot(scalesToTry,sitesDiffByScale,marker='.')
ax.set_ylabel('# of sites changed')
ax.set_xlabel('Laplace scale')
plt.show()

We see that by around scale = 5e-5, the single-site changes and short tracts have been smoothed over, and we choose this as our final smoothing scale.

**note** smoothing returns a copy of the dataset with the same order (i.e. HIs are not recomputed and data is not resorted). That way, comparisons can be made easily before and after smoothing. 


In [ ]:
dSmoothed = dThresh.smooth(5e-5)

We can compare before and after smoothing to see how it has affected our data:

In [ ]:
diem.plot_painting(dThresh.DMBC[9],dThresh.indNames)
diem.plot_painting(dSmoothed.DMBC[9],dSmoothed.indNames)

We can also see how smoothing has affected our HIs

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
ax.plot(dThresh.HIs,marker='.',color='b')
ax.plot(dSmoothed.computeHIs(),marker='.',color='r')
ax.set_ylabel('HI')
ax.set_xlabel('individual idx')
plt.show()

After smoothing, we can see that the hybrid indices have changed, and we now do a final sort of the data:

In [ ]:
dSmoothed.sort()
diem.plot_painting(dSmoothed.DMBC[9],dSmoothed.indNames)

# Constructing Contigs

Downstream analyses of diem output may focus on the lengths of tracts of shared ancestry across the gene flow barrier.

Here, we provide the function create_contig_matrix() that generates 'contigs' for each chromosome of each individual.

a 'contig' is a summary of the data.  It is the sequence of contiguous intervals that form a givien chromosome 

Once we have created the contigs, we can export per-individual contig files in bed formatting 

In [ ]:
dSmoothed.sort() # ensure that dSmoothed is sorted.
dSmoothed.create_contig_matrix()


In [ ]:
contingOutputDirectory = './output/contigs/'

diem.export_contigs_to_ind_bed_files(dSmoothed,contingOutputDirectory)


# Using Contigs & intervals

Within the diemtype objects, contigs are stored in the *contigMatrix*

the *contigMatrix* is an array wit shape nChrs x nIndividuals. and we can use splicing to access specific contigs. 

For chromosome indexed 1 and individual indexed 8:

In [ ]:
exampleContig = dSmoothed.contigMatrix[1,8]
exampleContig.__dict__

A contig stores the individual and chromosome names to which it belongs as well as a list of intervals and the number of intervals

In [ ]:
?exampleContig

In [ ]:
exampleContig.num_intervals

an **interval** is a contiguous segment of genome with the same ancestry.

It stores the chromosome and individual to which it belongs. the indices and positions of the left- and right-most markers,
and the state of the individual: 0, 1, 2, or 3.

In [ ]:
exampleInterval = exampleContig.intervals[0]
?exampleInterval

In [ ]:
exampleInterval.__dict__

We can get the interval's span easily:

In [ ]:
exampleInterval.span()

And we can get the intervals map span (on a unit scale) if we provide the length of the chromsome.

In [ ]:
exampleInterval.mapSpan(dSmoothed.chrLengths[1])

Here is the information about all the intervals that form the genotype of our example contig:

In [ ]:
[x.__dict__ for x in exampleContig.intervals]

We can plot the tract length distribution for all intervals with state '1'

# Plotting tract length distributions

In [ ]:
oneTracts = dSmoothed.get_intervals_of_state(1)
oneLengths = [x.span() for x in oneTracts]
fig,ax = plt.subplots()
ax.hist(oneLengths,color='m')
ax.set_ylabel('counts')
ax.set_xlabel('tract length')
plt.show()

We can plot the distribution of the different types of tracts together

In [ ]:
fig,ax = plt.subplots()
ax.hist([x.span() for x in dSmoothed.get_intervals_of_state(1)],bins = 50,color='m',alpha = 0.5)
ax.hist([x.span() for x in dSmoothed.get_intervals_of_state(2)],bins=50,color='y', alpha = 0.5)
ax.hist([x.span() for x in dSmoothed.get_intervals_of_state(3)],bins=50,color='teal', alpha = 0.5)
ax.set_ylabel('counts')
ax.set_xlabel('tract length')
plt.show()

# Advanced tract length distribution plotting

We can also subset individuals and chromosomes when getting intervavls of those types.

Suppose we want to compare the distribution of tract lengths on opposite sides of the barrier. By looking at the HIs of the smoothed data, we see there is one highly admixed individual, IF_1322, so we decide to exclude it here. 

We then identify the individuals clearly to the 'left (bottom)' and 'right (top)' sides of the barrier. 

In [ ]:
[[i,dSmoothed.indNames[i],dSmoothed.HIs[i]] for i in range(len(dSmoothed.indNames))]

In [ ]:
leftIndividualIndices = np.arange(8)
rightIndividualIndices = np.arange(9,20)
print(leftIndividualIndices)
print(rightIndividualIndices)

We can plot the spans of tracts of type '1' for individuals on the 'left' (blue) and 'right' (red) sides of the barrier, respectively. 

In [ ]:
state1SpansLeftSide = [x.span() for x in dSmoothed.get_intervals_of_state(1,individualSubset=leftIndividualIndices)]
state1SpansRightSide = [x.span() for x in dSmoothed.get_intervals_of_state(1,individualSubset=rightIndividualIndices)]

fig,ax = plt.subplots()
longestSpan = max(max(state1SpansLeftSide),max(state1SpansRightSide))
ax.hist(state1SpansLeftSide,bins = 100,range=(0,longestSpan),color = 'b',alpha = 0.5)
ax.hist(state1SpansRightSide, bins = 100,range=(0,longestSpan), color = 'r',alpha = 0.5)
ax.set_ylabel('counts')
ax.set_xlabel('tract length')
plt.show()

Similarly, we can plot the spans of type '3' comparing the left and right sides of the barrier

In [ ]:
state3SpansLeftSide = [x.span() for x in dSmoothed.get_intervals_of_state(3,individualSubset=leftIndividualIndices)]
state3SpansRightSide = [x.span() for x in dSmoothed.get_intervals_of_state(3,individualSubset=rightIndividualIndices)]

fig,ax = plt.subplots()
longestSpan = max(max(state3SpansLeftSide),max(state3SpansRightSide))
ax.hist(state3SpansLeftSide,bins = 100,range=(0,longestSpan),color = 'b',alpha = 0.5)
ax.hist(state3SpansRightSide, bins = 100,range=(0,longestSpan), color = 'r',alpha = 0.5)
ax.set_xlabel('tract length')
ax.set_ylabel('counts')
plt.show()

# Visualisation

Polarisation has not changed your data, only coloured it and measured it. The **GenomeSummaryPlot** shows these colours, and the effects of ignoring sites with low diagnostic index (DI).

Sort individuals by their hybrid index using the red button. Slide the DI slider to the right to focus on barrier relevant sites. If your data is large, you may need to _click_ at your desired DI value rather than sliding. Mouse-over a plot symbol to see which individual it refers to. You can save the plot using the click-on tool or keyboard shortcut S.

**TOUCHDOWN DI**: As you slide DI to the right, first Unpolarisable (U, grey line) data will be filtered out (blanks and genotypes involving third states). Next the span of the hybrid indices (red line) should start to increase (you may wish to hit the red button to re-sort by the new HI). One way of choosing a DI threshold is to move right until 'touchdown' where the hybrid index goes to zero on one side of the barrier, and 1 on the other. An outlier individual may touchdown first, way before others; in that case keep going until other have joined. There is a trade-off: the further to the right, the cleaner the retained data with respect to the barrier, but the fewer sites in the genome are being considered.

In [ ]:
diem.plots.GenomeSummaryPlot(dPol)

The effects of DI sliding may not be equal across chromosomes - sex chromosome are often more relevant to barriers. **GenomeMultiSummaryPlot** allows you to explore this by specifying a list of chromosome to summarise as though each is a genome, while still showing the global HI plot (cyan) over all chromosome as reference.

In [ ]:
diem.plots.GenomeMultiSummaryPlot(dPol,[0,1,2])

There are many ways to summarise genomes with respect to the sites diagnosing a barrier. The **GenomicDeFinetti** diagram uses HOM1 HET HOM2 frequencies to place genomes in a ternary plot. The colours are a blend of not just these proportions, but also the Unpolarisable proportion (as White). Skim-sequenced genomes will, for example, have lighter shades due to more missing data.

The background parabola is analogous to Hardy-Weinberg equilibrium for HOM1 HET HOM2 frequencies at a single site of a genome. In current context the parabola is the genomic expectation if admixture is proceeding with local HWE in a simlar fashion throughout the genome (ie with similar admixture proportions).

In [ ]:
diem.plots.GenomicDeFinettiPlot(dPol)

Again, the effects of DI sliding may not be equal across chromosomes or genome regions. **GenomicMultiDeFinettiPlot** allows you to explore this at the level of chromosomes from the ternary perspective by specifying a list of chromosome to summarise as though each is a genome.

In [ ]:
diem.plots.GenomicMultiDeFinettiPlot(dPol,range(3))

Another way of exploring the heterogeneity of barrier effects across the genome is to simply look at howmany sites are retained on each chromosome after DI sliding. **GenomicContributionsPlot** shows these contributions, and breaks them down by state: often sex chromsomes will contribute a lot, but with a lower HET proportion.

The resemblance to a graphical equaliser from an old stereo raises an analogy: sliding DI to the right, we see the channels (chromosomes) on which barrier signal is strongest.

In [ ]:
diem.plots.GenomicContributionsPlot(dPol)

It may be that admixture has produced an explosion of genome-types, such that genomes are all unique and their summaries change smoothly across the barrier. Alternatively, for eg a taxon of hybrid origin, a single along-genome blockwise admixture pattern may occur in multiple individuals. **diemPairsPlot** allows you to explore these possibilities through all pairwise comparison of the painted colours of the polarised genomes. 

The is a genomic O(n<sup>2</sup>) calculation... so choose one DI threshold at a time (maybe a touchdown DI) it may take a while before it plots.... be patient.

In the plot genomes are ordered simply by their hybrid index.

Blue squares indicate clusters of genomes which share detailed purple/teal long-genome pattern.

You may note the diagonal of self-to-self comparison is not always zero. The same distance function is applied for all comparisons (even self-to-self) in order to make it clear that, because _diem_ is ignorant of phase, comparison of HET genome tracts to HET genome tracts do **not** give distance zero. HOM1 vs HOM2 is distance 2. HET vs HET is (either 0 or 2 with phase unknown) => distance 1. (See Petruezela et al Mol Ecol 2025).

In [ ]:
diem.plots.diemPairsPlot(dPol,-100)

In [ ]:
diem.plots.diemMultiPairsPlot(dPol,[0,1,2,3,4,5,23],-100)

We mentioned 'detailed purple/teal long-genome pattern'. What does this look like? **diemIrisFromPlotPrep** allows us to see this detail for the whole genome, for all the genomes, all at once. 

Again this is a computationally intensive task, so first we run a prepatory step **diemPlotPrepFromBedMeta**, which loads the data you saved in the **Polarisation** section and prepares is for the big plot of everything...

Give your plot a (short) theme.
Set the correct files from **Polarisation**
Choose a (touchdown) DI threshold
Choose a number of 'pixels' (in multiple of 360... the plot is going to be a ring)
Choose the interval between ticks in bases
(We will come to smoothing later).

In [ ]:
MyFirstPlotPrep = diem.plots.diemPlotPrepFromBedMeta(
    plot_theme="tutorial example",
    bed_file_path=bedOutFile,
    meta_file_path=metaCorrected,
    di_threshold=-57,  
    genome_pixels=360*1,
    ticks=10000, # bases
    smooth=5e-5
)

Now we construct an Iris Plot from the prepared data:

Mouse-over and you genome/indidivual location will be displayed below the plot... browse your admixture features.

In [ ]:
diem.plots.diemIrisFromPlotPrep(MyFirstPlotPrep)

We can straighten out this curl to make a rectangular plot using **diemLongFromPlotPrep**:

You can choose which chromosomes to include.

Again this is browsable.

In [ ]:
diem.plots.diemLongFromPlotPrep(MyFirstPlotPrep)

You can limit the plot to a subset of chromosomes

In [ ]:
diem.plots.diemLongFromPlotPrep(MyFirstPlotPrep,range(3))

Or you can stack plots of each of your chromosomes:

In [ ]:
for i in range(3):
    diem.plots.diemLongFromPlotPrep(MyFirstPlotPrep,[i])

# Advanced Polarizing

When running diem, we may want to mask some individual and sites.  Here, masked sites/individuals do not *influence* the barrier (i.e. they don't contribute to the likelihood function). However, they *are* polarized during the EM step along with other sites and individuals. 

We have included two example exclusion files under the user_files subdirectory.

Here, we mask individuals that were sampled outside of france and spain.

We also (arbitrarily) mask chromosome 2 for this example.

Although we have masked both sites and individuals in this example, it possible to specify only one or the other. Any masks have been added to the raw data will be used in polarization. 

In [ ]:
# #### add information about individuals that do not influence polarization. We say 'excluded' but really mean 'masked'
sitesMaskedFile = './user_files/sites_masked.bed'
individualsMaskedFile = './user_files/individuals_masked.txt'

dRawWithMasks = dRaw.copy()

dRawWithMasks.add_individual_exclusions(individualsMaskedFile)
dRawWithMasks.add_site_exclusions(sitesMaskedFile)

In [ ]:
dRawWithMasks.siteExclusionsByChr[12]

In [ ]:
dPolWithMasks = dRawWithMasks.polarize()
dPolWithMasks.sort()

In [ ]:
diem.plot_painting(dPolWithMasks.DMBC[9])

fig,ax = plt.subplots(figsize=(6,4))
ax.plot(dPolWithMasks.HIs,marker='.',color='b')
ax.set_ylabel('HI')
ax.set_xlabel('individual idx')
plt.show()